# Sales Tracker Dashboard — DuckDB Edition

In [ ]:
import duckdb
import pandas as pd

# Native DuckDB in-memory connection
con = duckdb.connect(":memory:")
print("DuckDB version:", duckdb.__version__)

In [ ]:
# Load CSV directly into DuckDB table
con.execute("""
    CREATE TABLE sales AS
    SELECT * FROM read_csv(
        'dataset/data.csv',
        header = true,
        columns = {
            'Date'           : 'DATE',
            'BranchID'       : 'VARCHAR',
            'BranchName'     : 'VARCHAR',
            'ProductCategory': 'VARCHAR',
            'ProductName'    : 'VARCHAR',
            'Quantity'       : 'INTEGER',
            'Price'          : 'BIGINT',
            'TotalSales'     : 'BIGINT',
            'Salesperson'    : 'VARCHAR'
        }
    )
""")
total = con.execute("SELECT COUNT(*) FROM sales").fetchone()[0]
print(f"Loaded {total:,} rows into DuckDB table [sales]")

---
## Challenge 1: Data Cleaning

### 1.1 Initial Inspection

In [ ]:
# Schema
con.execute("DESCRIBE sales").df()

In [ ]:
# Sample rows
con.execute("SELECT * FROM sales LIMIT 5").df()

In [ ]:
# Date range & basic stats
con.execute("""
    SELECT
        COUNT(*)          AS total_rows,
        MIN(Date)         AS date_min,
        MAX(Date)         AS date_max,
        MIN(Quantity)     AS qty_min,
        MAX(Quantity)     AS qty_max,
        MIN(Price)        AS price_min,
        MAX(Price)        AS price_max,
        MIN(TotalSales)   AS sales_min,
        MAX(TotalSales)   AS sales_max
    FROM sales
""").df()

### 1.2 Date Filter: 2025-01-01 to 2026-03-31

In [ ]:
con.execute("""
    CREATE OR REPLACE TABLE sales_filtered AS
    SELECT * FROM sales
    WHERE Date BETWEEN '2025-01-01' AND '2026-03-31'
""")
n = con.execute("SELECT COUNT(*) FROM sales_filtered").fetchone()[0]
orig = con.execute("SELECT COUNT(*) FROM sales").fetchone()[0]
print(f"Before filter : {orig:,}")
print(f"After  filter : {n:,}")
print(f"Removed       : {orig - n:,}")

### 1.3 Anomaly Detection

In [ ]:
# 1. Duplicate rows (exact match)
con.execute("""
    SELECT COUNT(*) AS duplicate_rows
    FROM (
        SELECT *, COUNT(*) OVER (
            PARTITION BY Date, BranchID, ProductName,
                         Quantity, Price, TotalSales, Salesperson
        ) AS cnt
        FROM sales_filtered
    )
    WHERE cnt > 1
""").df()

In [ ]:
# Show duplicate row examples
con.execute("""
    SELECT *
    FROM (
        SELECT *, COUNT(*) OVER (
            PARTITION BY Date, BranchID, ProductName,
                         Quantity, Price, TotalSales, Salesperson
        ) AS cnt
        FROM sales_filtered
    )
    WHERE cnt > 1
    ORDER BY Date, BranchID, ProductName
    LIMIT 8
""").df()

In [ ]:
# 2. Negative values
con.execute("""
    SELECT
        SUM(CASE WHEN Quantity   < 0 THEN 1 ELSE 0 END) AS neg_quantity,
        SUM(CASE WHEN Price      < 0 THEN 1 ELSE 0 END) AS neg_price,
        SUM(CASE WHEN TotalSales < 0 THEN 1 ELSE 0 END) AS neg_totalsales,
        SUM(CASE WHEN Quantity < 0 OR Price < 0 OR TotalSales < 0
                 THEN 1 ELSE 0 END)                     AS total_negative_rows
    FROM sales_filtered
""").df()

In [ ]:
# Show negative rows
con.execute("""
    SELECT * FROM sales_filtered
    WHERE Quantity < 0 OR Price < 0 OR TotalSales < 0
    ORDER BY Date
""").df()

In [ ]:
# 3. Future dates (> 2026-02-21, today)
con.execute("""
    SELECT COUNT(*) AS future_date_rows
    FROM sales_filtered
    WHERE Date > '2026-02-21'
""").df()

In [ ]:
# Show future date rows
con.execute("""
    SELECT * FROM sales_filtered
    WHERE Date > '2026-02-21'
    ORDER BY Date
""").df()

In [ ]:
# 4. Price = 0 or TotalSales = 0 while Quantity > 0
con.execute("""
    SELECT COUNT(*) AS zero_price_sales_rows
    FROM sales_filtered
    WHERE (Price = 0 OR TotalSales = 0)
      AND Quantity > 0
""").df()

In [ ]:
# Show zero-price rows
con.execute("""
    SELECT * FROM sales_filtered
    WHERE (Price = 0 OR TotalSales = 0)
      AND Quantity > 0
""").df()

In [ ]:
# 5. Date gaps (missing dates in range)
con.execute("""
    WITH date_series AS (
        SELECT unnest(
            generate_series(DATE '2025-01-01', DATE '2026-03-31', INTERVAL 1 DAY)
        )::DATE AS d
    ),
    present AS (
        SELECT DISTINCT Date FROM sales_filtered
    )
    SELECT d AS missing_date
    FROM date_series
    WHERE d NOT IN (SELECT Date FROM present)
    ORDER BY d
""").df()

In [ ]:
# 6. TotalSales mismatch (Price * Quantity != TotalSales)
con.execute("""
    SELECT COUNT(*) AS mismatch_rows
    FROM sales_filtered
    WHERE Price * Quantity <> TotalSales
""").df()

### 1.4 Anomaly Summary Table

In [ ]:
con.execute("""
    SELECT 'Duplicate rows (exact match)'              AS anomaly_type,
           COUNT(*)                                    AS count
    FROM (
        SELECT *, COUNT(*) OVER (
            PARTITION BY Date, BranchID, ProductName,
                         Quantity, Price, TotalSales, Salesperson
        ) AS cnt FROM sales_filtered
    ) WHERE cnt > 1
    UNION ALL
    SELECT 'Negative values (Qty/Price/TotalSales)',
           SUM(CASE WHEN Quantity < 0 OR Price < 0 OR TotalSales < 0
                    THEN 1 ELSE 0 END)
    FROM sales_filtered
    UNION ALL
    SELECT 'Future dates (> 2026-02-21)',
           COUNT(*) FROM sales_filtered WHERE Date > '2026-02-21'
    UNION ALL
    SELECT 'Price/TotalSales = 0 with Qty > 0',
           COUNT(*) FROM sales_filtered
    WHERE (Price = 0 OR TotalSales = 0) AND Quantity > 0
    UNION ALL
    SELECT 'Date gaps (missing dates)',
           COUNT(*)
    FROM (
        WITH ds AS (
            SELECT unnest(generate_series(
                DATE '2025-01-01', DATE '2026-03-31', INTERVAL 1 DAY
            ))::DATE AS d
        )
        SELECT d FROM ds
        WHERE d NOT IN (SELECT DISTINCT Date FROM sales_filtered)
    )
    UNION ALL
    SELECT 'TotalSales mismatch (Price x Qty != TotalSales)',
           COUNT(*) FROM sales_filtered
    WHERE Price * Quantity <> TotalSales
""").df()

### 1.5 Data Cleaning — Remove Anomalies

In [ ]:
con.execute("""
    CREATE OR REPLACE TABLE sales_clean AS
    WITH deduped AS (
        -- Remove exact duplicates (keep first occurrence)
        SELECT DISTINCT *
        FROM sales_filtered
    )
    SELECT *
    FROM deduped
    WHERE
        -- Remove negative values
        Quantity   >= 0
        AND Price      >= 0
        AND TotalSales >= 0
        -- Remove future dates
        AND Date <= '2026-02-21'
        -- Remove zero price/sales with actual quantity
        AND NOT ((Price = 0 OR TotalSales = 0) AND Quantity > 0)
""")

before = con.execute("SELECT COUNT(*) FROM sales_filtered").fetchone()[0]
after  = con.execute("SELECT COUNT(*) FROM sales_clean").fetchone()[0]
print(f"Rows before cleaning : {before:,}")
print(f"Rows after  cleaning : {after:,}")
print(f"Total removed        : {before - after:,}")

In [ ]:
# Verify clean data
con.execute("""
    SELECT
        COUNT(*)        AS total_rows,
        MIN(Date)       AS date_min,
        MAX(Date)       AS date_max,
        MIN(Quantity)   AS qty_min,
        MIN(Price)      AS price_min,
        MIN(TotalSales) AS sales_min
    FROM sales_clean
""").df()

---
## Challenge 2: Trend Analysis

### 2.1 Most Profitable Branch

In [ ]:
con.execute("""
    SELECT
        ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) AS rank,
        BranchID,
        BranchName,
        SUM(TotalSales) AS total_revenue,
        SUM(Quantity)   AS total_qty,
        COUNT(*)        AS transactions
    FROM sales_clean
    GROUP BY BranchID, BranchName
    ORDER BY total_revenue DESC
""").df()

### 2.2 Best-Selling Products

In [ ]:
con.execute("""
    SELECT
        ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) AS rank,
        ProductCategory,
        ProductName,
        SUM(Quantity)   AS total_qty,
        SUM(TotalSales) AS total_revenue
    FROM sales_clean
    GROUP BY ProductCategory, ProductName
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

In [ ]:
# Revenue by category
con.execute("""
    SELECT
        ProductCategory,
        SUM(TotalSales)                           AS total_revenue,
        SUM(Quantity)                             AS total_qty,
        ROUND(100.0 * SUM(TotalSales) /
              SUM(SUM(TotalSales)) OVER (), 2)    AS pct_revenue
    FROM sales_clean
    GROUP BY ProductCategory
    ORDER BY total_revenue DESC
""").df()

### 2.3 Seasonal & Time Trends

In [ ]:
# Monthly trend
con.execute("""
    SELECT
        DATE_TRUNC('month', Date)  AS year_month,
        SUM(TotalSales)            AS total_revenue,
        SUM(Quantity)              AS total_qty,
        COUNT(*)                   AS transactions
    FROM sales_clean
    GROUP BY year_month
    ORDER BY year_month
""").df()

In [ ]:
# Day of week trend
con.execute("""
    SELECT
        DAYOFWEEK(Date)  AS dow_num,
        DAYNAME(Date)    AS day_name,
        SUM(TotalSales)  AS total_revenue,
        SUM(Quantity)    AS total_qty,
        COUNT(*)         AS transactions
    FROM sales_clean
    GROUP BY dow_num, day_name
    ORDER BY dow_num
""").df()

In [ ]:
# Salesperson performance
con.execute("""
    SELECT
        ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) AS rank,
        Salesperson,
        SUM(TotalSales) AS total_revenue,
        SUM(Quantity)   AS total_qty,
        COUNT(*)        AS transactions
    FROM sales_clean
    GROUP BY Salesperson
    ORDER BY total_revenue DESC
""").df()

In [ ]:
# Branch x Category heatmap data
con.execute("""
    SELECT
        BranchName,
        ProductCategory,
        SUM(TotalSales) AS total_revenue
    FROM sales_clean
    GROUP BY BranchName, ProductCategory
    ORDER BY BranchName, total_revenue DESC
""").df()

### 2.4 Insights

In [ ]:
# Quick insight summary
top_branch = con.execute("""
    SELECT BranchName, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY BranchName ORDER BY rev DESC LIMIT 1
""").fetchone()

top_product = con.execute("""
    SELECT ProductName, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY ProductName ORDER BY rev DESC LIMIT 1
""").fetchone()

top_day = con.execute("""
    SELECT DAYNAME(Date) AS d, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY d ORDER BY rev DESC LIMIT 1
""").fetchone()

peak_month = con.execute("""
    SELECT DATE_TRUNC('month', Date) AS m, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY m ORDER BY rev DESC LIMIT 1
""").fetchone()

print("=== KEY INSIGHTS ===")
print(f"Most profitable branch : {top_branch[0]} (Rp {top_branch[1]:,.0f})")
print(f"Best-selling product   : {top_product[0]} (Rp {top_product[1]:,.0f})")
print(f"Best day of week       : {top_day[0]}")
print(f"Peak month             : {str(peak_month[0])[:7]} (Rp {peak_month[1]:,.0f})")

---
## Challenge 3: Final Numbers

In [ ]:
# Angka kunci setelah data cleaning — harus EXACT MATCH
result = con.execute("""
    SELECT
        SUM(Quantity)            AS quantity_total,
        SUM(TotalSales)          AS totalsales_total,
        COUNT(DISTINCT Salesperson) AS salesperson_total
    FROM sales_clean
""").fetchone()

print("=" * 48)
print(f"  Quantity Total     : {result[0]:,}")
print(f"  TotalSales Total   : Rp {result[1]:,.0f}")
print(f"  Salesperson Total  : {result[2]}")
print("=" * 48)